In [ ]:
from glob import glob
import numpy as np
import pandas as pd
from scipy import stats
import os

from analysis_scripts import second_harm_tools
from analysis_scripts.analyzs_many import get_sea_urchin

In [ ]:
sea_urchins = glob(os.path.join("..", "data", "sea_urchin_darkfield","*.h5"))


In [ ]:
len(sea_urchins)

In [ ]:
defaults =  {"t_range": slice(None, None)}
params = {}


results = []
spectrums = []
for exp in sea_urchins:
    param = params.get(exp, defaults)
    if param.get("skip"):
        continue
    a = get_sea_urchin(exp, rel_s=0.9, **param)
    if a.last_valid_index() < 0.1:
        continue
    res_sea, spectrum = second_harm_tools.analyse_shape(a, NFFT=125, pca=True, smoothing=58)
    res_sea.index.name = "time"
    res_sea.reset_index(inplace=True)
    res_sea["exp"] = exp
    results.append(res_sea)
    spectrums.append(spectrum)
    
    
res_seas = pd.concat(results)


In [ ]:
5*1./res_seas.dropna().omega.mean()*500

In [ ]:
res_seas["phi_vel_normed_s_c0"] = res_seas.phi_vel_normed_s/(res_seas.C0**2)
res_seas["c2_plus_cm"] = 0.04*res_seas.C2_SIN + 0.36*res_seas.kappa_mean_s

In [ ]:
res = []
for spectrum in spectrums:
    res.append((spectrum[::100,:,0].flatten(), np.sqrt(spectrum[::100,:,1].flatten())))
xx = np.hstack(list(zip(*res))[0])
yy = np.hstack(list(zip(*res))[1])
    

In [ ]:
power, bin_edge, nbins = stats.binned_statistic(xx, yy,
                                                bins=np.arange(0.05,5.,0.1), statistic="mean")
power_std, bin_edge, nbins = stats.binned_statistic(xx, yy,
                                                bins=np.arange(0.05,5.,0.1), statistic="std")

freq = (bin_edge[1:]+bin_edge[:-1])/2


In [ ]:
res_seas.to_hdf(os.path.join("..", "c2_kappa_mean_rotation.h5"), 'sea_urchins')
np.savetxt(os.path.join("..", "sea_urchin_spectrum.csv"),  np.vstack([freq, power, power_std]).T)